## 1. Setup - Ridge Regression 
Cost function: $$\begin{align} J(\theta) &= \frac{1}{m}\sum_{i=1}^m(\theta^Tx_i- y_i)^2 + \lambda  \theta^T\theta, \; x_i, \theta \in \mathrm{R}^{d+1} \\
&= \frac{1}{m}[(X\theta-y)^{T} (X\theta-y)] + \lambda \theta^T\theta, \; X \in \mathrm{R}^{m\times(d+1)}
\end{align}$$
Gradient of cost function: $$\begin{align} 
\nabla J(\theta) = \frac{1}{m}[2X^{T}X\theta-2X^{T}y]+2\lambda\theta
\end{align}$$
When $m$ (number of data points) is large, computing a single gradient can take a very long time! 

### Stochastic Gradient Descent 

When the cost function takes in the form of $J(\theta) = \frac{1}{m}\sum_{i=1}^mf_i(\theta)$, we can show that $\nabla f_i(\theta)$ for some data point $i$ chosen uniformly at random from the data set is an unbiased estimator of $\nabla J(\theta)$. Instead of taking a gradient step of $-\nabla J(\theta)$, we instead take $-\nabla f_i(\theta)$, this is called **Stochastic Gradient Descent**

$$J(\theta) = \frac{1}{m}\sum_{i=1}^mf_i(\theta), \text{ where } f_i(\theta) = (\theta^Tx_i- y_i)^2 + \lambda  \theta^T\theta$$
$$\nabla f_{i}(\theta) = 2x_i^{T}(x_i\theta-y_i)+2\lambda\theta $$
$$E(\nabla f_{i}(\theta)) = \frac{1}{m}\sum_{i=1}^m(\nabla f_{i}(\theta)) = \nabla J(\theta)$$

parameter update rule: 
$$\theta \leftarrow \theta - \eta ( 2x_i^{T}(x_i\theta-y_i)+2\lambda\theta)$$

In [ ]:
def compute_regularized_square_loss_gradient(X, y, theta, lambda_reg):
    """
    Compute the gradient of L2-regularized square loss function given X, y and theta

    Args:
        X - the feature vector, 2D numpy array of size (num_instances, num_features)
        y - the label vector, 1D numpy array of size (num_instances)
        theta - the parameter vector, 1D numpy array of size (num_features)
        lambda_reg - the regularization coefficient

    Returns:
        grad - gradient vector, 1D numpy array of size (num_features)
    """
    
    First_term = np.dot(np.dot(X.T,X),theta)
    Second_term = (X.T).dot(y)
    num_instances = X.shape[0]
    grad = (First_term-Second_term)*2/num_instances + theta*2*lambda_reg
    
    return grad

In [2]:
def stochastic_grad_descent(X, y, alpha=0.1, lambda_reg=1, num_iter=1000):
    """
    In this question you will implement stochastic gradient descent with a regularization term

    Args:
        X - the feature vector, 2D numpy array of size (num_instances, num_features)
        y - the label vector, 1D numpy array of size (num_instances)
        alpha - string or float. step size in gradient descent
                NOTE: In SGD, it's not always a good idea to use a fixed step size. Usually it's set to 1/sqrt(t) or 1/t
                if alpha is a float, then the step size in every iteration is alpha.
                if alpha == "1/sqrt(t)", alpha = 1/sqrt(t)
                if alpha == "1/t", alpha = 1/t
        lambda_reg - the regularization coefficient
        num_iter - number of epochs (i.e number of times to go through the whole training set)

    Returns:
        theta_hist - the history of parameter vector, 3D numpy array of size (num_iter, num_instances, num_features)
        loss hist - the history of **regularized loss function** vector, 2D numpy array of size(num_iter, num_instances)
    """
    num_instances, num_features = X.shape[0], X.shape[1]
    theta = np.ones(num_features) #Initialize theta

    theta_hist = np.zeros((num_iter, num_instances, num_features))  #Initialize theta_hist
    loss_hist = np.zeros((num_iter, num_instances)) #Initialize loss_hist

    t = 1
    arr = np.arange(num_instances)
    np.random.shuffle(arr)
    for i in range(num_iter): #each epoch
        np.random.shuffle(arr) #shuffle once every epoch
        for j in arr:
            grad = compute_regularized_square_loss_gradient(X[j:j+1,:], y[j:j+1], theta, lambda_reg)
            if type(alpha) == float:
                alpha_new = alpha
            elif alpha == "1/t":
                alpha_new = 1/(t+10)
            elif alpha == "1/sqrt(t)":
                alpha_new = 1/((t+10)**0.5)
            theta = theta - alpha_new * grad
            loss = compute_square_loss(X,y,theta)
            
            theta_hist[i,j] = theta
            loss_hist[i,j] = loss
            t+=1

    return theta_hist, loss_hist

SGD in Sklearn: https://scikit-learn.org/stable/modules/sgd.html#id1